In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



url = "https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/comprehensive_supply_chain_data_with_weather_spike.csv"
df = pd.read_csv(url, parse_dates=['Timestamp'], index_col='Timestamp')



features = ['Inventory Levels', 'Lead Time (days)', 'Supplier Reliability (%)',
            'Economic Indicator (GDP Growth %)', 'Marketing Spend ($)',
            'Price Fluctuation (%)', 'Competitor Pricing ($)', 'Customer Sentiment',
            'Temperature (°C)', 'Weather Spike', 'Transportation Costs ($)',
            'Stockouts (Binary)', 'Promotions and Discounts (%)', 'Product Quality (%)']

X = df[features]
y = df['Demand']




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)



param_grid = {
'n_estimators': [100, 200, 300],
'max_depth': [4, 6, 8],
'learning_rate': [0.01, 0.05, 0.1],
'subsample': [0.7, 0.8, 0.9]
}



grid_search = GridSearchCV(xgb_model, param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
grid_search.fit(X_train_scaled, y_train)



best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)



best_xgb_model = xgb.XGBRegressor(objective='reg:squarederror',
                                  n_estimators=best_params['n_estimators'],
                                  max_depth=best_params['max_depth'],
                                  learning_rate=best_params['learning_rate'],
                                  subsample=best_params['subsample'],
                                  random_state=42)
best_xgb_model.fit(X_train_scaled, y_train)



y_pred = best_xgb_model.predict(X_test_scaled)



mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2 Score): {r2}")


plt.figure(figsize=(15, 5))



plt.subplot(1, 2, 1)
plt.plot(y_test.values, label='Actual Demand', color='blue')
plt.plot(y_pred, label='Predicted Demand', color='red')
plt.title('Figure 8.11: Actual vs Predicted Demand')
plt.legend()



residuals = y_test - y_pred
plt.subplot(1, 2, 2)
plt.plot(residuals.values, label='Residuals', color='green')
plt.title('Figure 8.12: Residuals')
plt.legend()

plt.tight_layout()
plt.show()




import shap
explainer = shap.Explainer(best_xgb_model, X_train_scaled)
shap_values = explainer(X_train_scaled)
shap.summary_plot(shap_values, X_train, plot_type='bar')




from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(best_xgb_model, X_test_scaled, y_test, n_repeats=10, random_state=42)
sorted_idx = perm_importance.importances_mean.argsort()

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), [features[i] for i in sorted_idx])
plt.xlabel('Permutation Importance')
plt.title(' Figure 8.14: Feature Importance for XGBoost Model')
plt.show()
